IMDB MOVIE REVIEW SENTIMENT ANALYSIS WITH TENSORFLOW AND BERT

1) Connecting Kaggle IMDB Review Dataset By Using Kaggle API

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

In [5]:
#files.upload()

In [6]:
! mkdir ~/.kaggle

In [7]:
! cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
! kaggle datasets list

ref                                                             title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                   5MB  2022-11-13 15:47:17          11204        328  1.0              
michals22/coffee-dataset                                        Coffee dataset                                    24KB  2022-12-15 20:02:12           2076         58  1.0              
thedevastator/jobs-dataset-from-glassdoor                       Salary Prediction                                  3MB  2022-11-16 13:52:31           7006        152  1.0              
thedevastator/unlock-profits-with-e-commerce-sales-data         E-Commerce 

In [10]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 35% 9.00M/25.7M [00:00<00:00, 90.1MB/s]
100% 25.7M/25.7M [00:00<00:00, 163MB/s] 


In [11]:
! unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


2) Importing necessary packages

In [12]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 71.3 MB/s 
     |████████████████████████████████| 182 kB 61.2 MB/s 


In [19]:
# Regular imports
import numpy as np
import pandas as pd
import tqdm # for progress bar
import math
import random
import re

from sklearn.model_selection import train_test_split

# Tensorflow Import
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer


pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)


In [20]:
movie_reviews = pd.read_csv("IMDB Dataset.csv")

In [21]:
movie_reviews.head(20)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regard...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the ref...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Paren...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler's play about the same theme, the director transfers the action t...",positive
5,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more l...",positive
6,I sure would like to see a resurrection of a up dated Seahunt series with the tech they have today it would bring back the kid excitement in me.I grew up on black and white TV and Seahunt with Gunsmoke were my hero's every week.You have my vote for a comeback of a new sea hunt.We need a change of pace in TV and this would work for a world of under water adventure.Oh by the way thank you for an...,positive
7,"This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 or 8 years were brilliant, but things dropped off after that. By 1990, the show was not really funny anymore, and it's continued its decline further to the complete waste of time it is today.<br /><br />It's truly disgraceful how far this show has fallen. The writing is painfully bad, the performance...",negative
8,"Encouraged by the positive comments about this film on here I was looking forward to watching this film. Bad mistake. I've seen 950+ films and this is truly one of the worst of them - it's awful in almost every way: editing, pacing, storyline, 'acting,' soundtrack (the film's only song - a lame country tune - is played no less than four times). The film looks cheap and nasty and is boring in t...",negative
9,"If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it.<br /><br />Great Camp!!!",positive


To be able to use the text, we have to prepare it accordingly. In the first step, we create a function that removes the line breaks and other HTML leftovers from the text. In this step, we also filter out other text impurities using Regular Expressions.

In [22]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
  return TAG_RE.sub('', text)

def preprocess_tags(sen):
  # Removing html tags
  sentence = remove_tags(sen)

  # Remove punctuations and numbers
  sentence = re.sub('[^a-zA-Z]',' ', sentence)

  # Single character removal
  sentence = re.sub(r's+[a-zA-Z]s+',' ', sentence)

  # Removing multiple spaces
  sentence = re.sub(r's+', ' ', sentence)

  return sentence


In [23]:
movie_reviews['review'] = movie_reviews['review'].apply(preprocess_tags)
movie_reviews['sentiment'] = movie_reviews['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [24]:
# ds = tf.data.Dataset.from_tensor_slices((
#     dict(movie_reviews['review']),
#     movie_reviews['sentiment'],

# ))

ds = (
    tf.data.Dataset.from_tensor_slices(
        (
            movie_reviews['review'].values,
            tf.cast(movie_reviews['sentiment'].values, tf.int32)
        )
    )
)

In [25]:
ds.__len__()

<tf.Tensor: shape=(), dtype=int64, numpy=50000>

Now we need to apply BERT tokenizer to use pre-trained tokenizer and then prepare data for BERT model.

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [27]:
max_length = 512
def convert_to_feature(review):
    return tokenizer.encode_plus(review,
                    add_special_tokens=True,
                    max_length=max_length,
                    pad_to_max_length=True,
                    return_attention_mask=True)

Transforming raw data to suitable form for BERT Model

In [28]:
def map_to_dict(input_ids,attention_mask, token_type_ids, label):
    return {
        "input_ids":input_ids,
        "attention_mask":attention_mask,
        "token_type_ids":token_type_ids
    },label

In [29]:
def encode_reviews(ds, limit=-1):
    input_ids_list=[]
    token_type_ids_list=[]
    attention_mask_list=[]
    label_list=[]
    
    if limit >0:
        ds = ds.take(limit)
    
    for review, label in tfds.as_numpy(ds):
        bert_input = convert_to_feature(review.decode())
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append(label)
    
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_to_dict)
    


Spliting dataset as train and test.

In [30]:
#ds_size = print([i for i,_ in enumerate(ds)][-1] + 1)
ds_size = int(ds.__len__())
print(ds_size)

50000


In [31]:
train_size = int(ds_size*0.8)
test_size = int(ds_size*0.2)
print(train_size)
print(test_size)

ds_train = ds.take(train_size)
ds_test = ds.skip(train_size).take(test_size)

40000
10000


Now we need to create out train and test datasets.

In [32]:
batch_size=6
ds_train_encoded = encode_reviews(ds_train).shuffle(10000).batch(batch_size)
ds_test_encoded = encode_reviews(ds_test).shuffle(10000).batch(batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Now we need to initialize BERT model for sentiment analysis.

In [34]:
from transformers import TFBertForSequenceClassification
learning_rate = 2e-5
epoch=2
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-8)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now training our sentiment analysis BERT model.

In [35]:
bert_history=model.fit(ds_train_encoded, validation_data=ds_test_encoded, epochs=epoch)

Epoch 1/2
6667/6667 [==============================] - 5533s 827ms/step - loss: 0.2949 - accuracy: 0.8740 - val_loss: 0.2540 - val_accuracy: 0.8974
Epoch 2/2
6667/6667 [==============================] - 5509s 826ms/step - loss: 0.1637 - accuracy: 0.9388 - val_loss: 0.3351 - val_accuracy: 0.8899


Testing our model with an random example.

In [36]:
test_sentence = "This is a really good movie. I loved it and will watch again"
predict_input = tokenizer.encode(test_sentence,
                                truncation=True,
                                padding=True,
                                return_tensors='tf')

In [42]:
tf_output = model.predict(predict_input)
tf_prediction = tf.nn.softmax(tf_output.logits, axis=1)
labels=['Negative','Positive']
label=tf.argmax(tf_prediction, axis=1)
label=label.numpy()
print(labels[label[0]])

1/1 [==============================] - 0s 43ms/step
Positive
